In [ ]:
import modal
import os
from dataclasses import dataclass
from pathlib import Path
from fastapi import FastAPI
from fastapi.responses import FileResponse
from modal import (
    Image,
    Mount,
    Secret,
    Volume,
    asgi_app,
    enter,
    method)

### Build up the environment
app = modal.App(name="dumpling-dreambooth-app")

image = Image.debian_slim(python_version="3.10").pip_install(
    "accelerate==0.27.2",
    "datasets~=2.13.0",
    "ftfy~=6.1.0",
    "gradio~=4.29.0",
    "smart_open~=6.4.0",
    "transformers~=4.38.1",
    "torch~=2.2.0",
    "torchvision~=0.16",
    "triton~=2.2.0",
    "peft==0.7.0",
    "wandb==0.16.3",)

### Downloading scripts and installing a git repo with `run_commands`
GIT_SHA = (
    "abd922bd0c43a504e47eca2ed354c3634bd00834"  # specify the commit to fetch)

image = (
    image.apt_install("git")
    # Perform a shallow fetch of just the target `diffusers` commit, checking out
    # the commit in the container's home directory, /root. Then install `diffusers`
    .run_commands(
        "cd /root && git init .",
        "cd /root && git remote add origin https://github.com/huggingface/diffusers",
        f"cd /root && git fetch --depth=1 origin {GIT_SHA} && git checkout {GIT_SHA}",
        "cd /root && pip install -e .",
    ))

### Configuration with `dataclass`es
with image.imports():
    from fastapi import Response
    import io
    import os

@dataclass
class SharedConfig:
    """Configuration information shared across project components."""
    # The instance name is the "proper noun" we're teaching the model
    instance_name: str = "Dumpling"
    # That proper noun is usually a member of some class (person, bird),
    # and sharing that information with the model helps it generalize better.
    class_name: str = "Little Black Cat with Golden Eyes"
    # identifier for pretrained models on Hugging Face
    model_name: str = "stabilityai/stable-diffusion-xl-base-1.0"
    vae_name: str = "madebyollin/sdxl-vae-fp16-fix"  # required for numerical stability in fp16

### Downloading weights with `run_function`
def download_models():
    import torch
    from diffusers import AutoencoderKL, DiffusionPipeline
    from transformers.utils import move_cache

    config = SharedConfig()

    DiffusionPipeline.from_pretrained(
        config.model_name,
        vae=AutoencoderKL.from_pretrained(
            config.vae_name, torch_dtype=torch.float16
        ),
        torch_dtype=torch.float16,
    )
    move_cache()

image = image.run_function(download_models)

### Storing data generated by our app with `modal.Volume`
volume = Volume.from_name(
    "dreambooth-finetuning-volume-v9", create_if_missing=True)
MODEL_DIR = "/model"

checkpoint_dir = None # i added this and below for automated inference by checkpoint
@dataclass
class CkptConfig:
    """Set checkpoint directory"""
    checkpoint_dir: str = None

### Load finetuning dataset
def load_images(image_urls: list[str]) -> Path:
    import PIL.Image
    from smart_open import open

    img_path = Path("/img")

    img_path.mkdir(parents=True, exist_ok=True)
    for ii, url in enumerate(image_urls):
        with open(url, "rb") as f:
            image = PIL.Image.open(f)
            image.save(img_path / f"{ii}.png")
    print(f"{ii + 1} images loaded")

    return img_path

### Finetuning with Hugging Face 🧨 Diffusers and Accelerate
USE_WANDB = True # you can turn it to False if not using Weights & Biases

@dataclass
class TrainConfig(SharedConfig):
    """Configuration for the finetuning step."""

    # training prompt looks like `{PREFIX} {INSTANCE_NAME} the {CLASS_NAME} {POSTFIX}`
    prefix: str = "a photo of"
    postfix: str = ""

    # locator for plaintext file with urls for images of target instance
    instance_example_urls_file: str = str(
        Path(__file__).parent / "instance_example_urls.txt"
    )

    # Hyperparameters/constants from the huggingface training example
    resolution: int = 1024
    train_batch_size: int = 4 #tried 2, dumpling had 5 legs
    gradient_accumulation_steps: int = 1
    learning_rate: float = 4e-4 #tried 4e-6 pic results didn't look very good altho loss trend very similar to other runs
    lr_scheduler: str = "constant"
    lr_warmup_steps: int = 0
    max_train_steps: int = 500 #80 before
    checkpointing_steps: int = 10
    seed: int = 117


@app.function(
    image=image,
    gpu="A100",  # fine-tuning is VRAM-heavy and requires an A100 GPU
    volumes={MODEL_DIR: volume},  # stores fine-tuned model
    timeout=3600,  # 30 minutes
    secrets=[Secret.from_name("my-wandb-secret")] if USE_WANDB else [],
)

def train(instance_example_urls):
    import subprocess

    from accelerate.utils import write_basic_config

    config = TrainConfig()

    # load data locally
    img_path = load_images(instance_example_urls)

    # set up hugging face accelerate library for fast training
    write_basic_config(mixed_precision="fp16")

    # define the training prompt
    instance_phrase = f"{config.instance_name} the {config.class_name}"
    prompt = f"{config.prefix} {instance_phrase} {config.postfix}".strip()

    # the model training is packaged as a script, so we have to execute it as a subprocess, which adds some boilerplate
    def _exec_subprocess(cmd: list[str]):
        """Executes subprocess and prints log to terminal while subprocess is running."""
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
        )
        with process.stdout as pipe:
            for line in iter(pipe.readline, b""):
                line_str = line.decode()
                print(f"{line_str}", end="")

        if exitcode := process.wait() != 0:
            raise subprocess.CalledProcessError(exitcode, "\n".join(cmd))

    # run training -- see huggingface accelerate docs for details
    print("launching dreambooth training script")
    _exec_subprocess(
        [
            "accelerate",
            "launch",
            "examples/dreambooth/train_dreambooth_lora_sdxl.py",
            "--mixed_precision=fp16",  # half-precision floats most of the time for faster training
            f"--pretrained_model_name_or_path={config.model_name}",
            f"--pretrained_vae_model_name_or_path={config.vae_name}",  # required for numerical stability in fp16
            f"--instance_data_dir={img_path}",
            f"--output_dir={MODEL_DIR}",
            f"--instance_prompt={prompt}",
            f"--resolution={config.resolution}",
            f"--train_batch_size={config.train_batch_size}",
            f"--gradient_accumulation_steps={config.gradient_accumulation_steps}",
            f"--learning_rate={config.learning_rate}",
            f"--lr_scheduler={config.lr_scheduler}",
            f"--lr_warmup_steps={config.lr_warmup_steps}",
            f"--max_train_steps={config.max_train_steps}",
            f"--checkpointing_steps={config.checkpointing_steps}",
            f"--seed={config.seed}",  # increased reproducibility by seeding the RNG
        ]
        + (
            [
                "--report_to=wandb",
                f"--validation_prompt={prompt} in space",  # simple test prompt
                f"--validation_epochs={config.max_train_steps // 10}",
            ]
            if USE_WANDB
            else []
        ),
    )
    # The trained model information has been output to the volume mounted at `MODEL_DIR`.
    # To persist this data for use in our web app, we 'commit' the changes
    # to the volume.
    volume.commit()

### Running our model (train & subsequently inference)
@app.local_entrypoint()
def run():
    with open(TrainConfig().instance_example_urls_file) as f:
        instance_example_urls = [line.strip() for line in f.readlines()]
    train.remote(instance_example_urls)

@app.cls(image=image, gpu="A10G", volumes={MODEL_DIR: volume})
class Model:
    @enter()
    def load_model(self):
        import torch
        from diffusers import AutoencoderKL, DiffusionPipeline

        config = TrainConfig()
        ckpt = CkptConfig()
        print(f"checkpoint directory: {ckpt.checkpoint_dir}")

        # Reload the modal.Volume to ensure the latest state is accessible.
        volume.reload()

        # set up a hugging face inference pipeline using our model
        pipe = DiffusionPipeline.from_pretrained(
            config.model_name,
            vae=AutoencoderKL.from_pretrained(
                config.vae_name, torch_dtype=torch.float16
            ),
            torch_dtype=torch.float16,
        ).to("cuda")
        pipe.load_lora_weights(ckpt.checkpoint_dir) # was pipe.load_lora_weights(MODEL_DIR)
        self.pipe = pipe
    
    def _inference(self, text, num_inference_steps: int=25, guidance_scale: float=7.5):
        image = self.pipe(
            text,
            num_inference_steps=num_inference_steps,
            guidance_scale=num_inference_steps,
        ).images[0]
        byte_stream=io.BytesIO()
        image.save(byte_stream,format="JPEG")
        return byte_stream

    @modal.web_endpoint(docs=True)
    def web_inference(self, text:str, num_inference_steps: int=25, guidance_scale: float=7.5):
        return Response(
            content=self._inference(text, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).getvalue(),
            media_type="image/jpeg",
        )


### To train, run the following command in terminal: "modal run name_of_this_file.py"
### To inference, run: "modal serve name_of_this_file.py" then open up the URL link, add \docs to test out different prompts
